In [1]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


PyTorch version: 2.9.0+cu126
CUDA available: True
Using device: cuda


In [1]:
# Step 2.1 - Dataset

texts = [
    "Experienced Python developer with knowledge of data structures",
    "Worked with pandas numpy and machine learning models",
    "Built deep learning models using pytorch and tensorflow",
    "Designed responsive websites using HTML CSS JavaScript",
    "Developed REST APIs using FastAPI and Django",
    "Performed data analysis and visualization using matplotlib",
    "Implemented CNNs and NLP models for AI applications",
    "Created frontend applications using React and Bootstrap"
]

labels = [
    0,  # Software Development
    1,  # Data Science
    2,  # AI / ML
    3,  # Web Development
    0,
    1,
    2,
    3
]

print("Total samples:", len(texts))


Total samples: 8


In [2]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts).toarray()

print("Shape of feature matrix:", X.shape)


Shape of feature matrix: (8, 45)


In [3]:
print(vectorizer.get_feature_names_out())


['ai' 'analysis' 'and' 'apis' 'applications' 'bootstrap' 'built' 'cnns'
 'created' 'css' 'data' 'deep' 'designed' 'developed' 'developer' 'django'
 'experienced' 'fastapi' 'for' 'frontend' 'html' 'implemented'
 'javascript' 'knowledge' 'learning' 'machine' 'matplotlib' 'models' 'nlp'
 'numpy' 'of' 'pandas' 'performed' 'python' 'pytorch' 'react' 'responsive'
 'rest' 'structures' 'tensorflow' 'using' 'visualization' 'websites'
 'with' 'worked']


In [4]:
import torch

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(labels, dtype=torch.long)

print("Feature tensor shape:", X_tensor.shape)
print("Label tensor shape:", y_tensor.shape)


Feature tensor shape: torch.Size([8, 45])
Label tensor shape: torch.Size([8])


In [5]:
import torch.nn as nn

class ResumeClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(ResumeClassifier, self).__init__()

        # Layer 1: Input → Hidden
        self.fc1 = nn.Linear(input_size, 64)

        # Activation
        self.relu = nn.ReLU()

        # Layer 2: Hidden → Output
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        # Forward pass defines computation graph
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


In [6]:
input_size = X_tensor.shape[1]
num_classes = 4

model = ResumeClassifier(input_size, num_classes)
print(model)


ResumeClassifier(
  (fc1): Linear(in_features=45, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=4, bias=True)
)


In [7]:
total_params = sum(p.numel() for p in model.parameters())
print("Total trainable parameters:", total_params)


Total trainable parameters: 3204


In [8]:
outputs = model(X_tensor)

print("Output shape:", outputs.shape)
print(outputs)


Output shape: torch.Size([8, 4])
tensor([[-0.2117,  0.1132,  0.0691,  0.2786],
        [-0.1978,  0.1119,  0.0697,  0.0572],
        [-0.3170,  0.1982,  0.0307, -0.0005],
        [-0.2371,  0.1875, -0.0553,  0.1371],
        [-0.2571,  0.2685, -0.0422,  0.0527],
        [-0.3312,  0.2422,  0.0262,  0.0910],
        [-0.1951,  0.0458,  0.0267,  0.0436],
        [-0.2590,  0.1732,  0.0888,  0.0593]], grad_fn=<AddmmBackward0>)


In [9]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()


In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [11]:
# Forward pass
outputs = model(X_tensor)

# Compute loss
loss = criterion(outputs, y_tensor)
print("Initial loss:", loss.item())

# Backward pass
optimizer.zero_grad()
loss.backward()

# Inspect gradients of first layer
print("Gradient of fc1 weights (sample):")
print(model.fc1.weight.grad[0][:5])

# Update weights
optimizer.step()


Initial loss: 1.3909013271331787
Gradient of fc1 weights (sample):
tensor([ 0.0108,  0.0000,  0.0088, -0.0106,  0.0139])


In [12]:
epochs = 50

for epoch in range(epochs):
    # 1. Forward pass
    outputs = model(X_tensor)

    # 2. Compute loss
    loss = criterion(outputs, y_tensor)

    # 3. Reset gradients
    optimizer.zero_grad()

    # 4. Backpropagation
    loss.backward()

    # 5. Update weights
    optimizer.step()

    # Print progress
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [10/50], Loss: 1.2597
Epoch [20/50], Loss: 1.1320
Epoch [30/50], Loss: 0.9954
Epoch [40/50], Loss: 0.8426
Epoch [50/50], Loss: 0.6792


In [13]:
def predict(text):
    model.eval()  # evaluation mode

    vec = vectorizer.transform([text]).toarray()
    tensor = torch.tensor(vec, dtype=torch.float32)

    with torch.no_grad():  # no gradient tracking
        output = model(tensor)
        _, predicted = torch.max(output, 1)

    return predicted.item()


In [14]:
label_map = {
    0: "Software Development",
    1: "Data Science",
    2: "AI / ML",
    3: "Web Development"
}

test_sentences = [
    "I have experience in neural networks and deep learning",
    "Skilled in pandas, numpy, and data visualization",
    "Built REST APIs and backend services",
    "Developed responsive UI using HTML CSS and JavaScript"
]

for text in test_sentences:
    pred = predict(text)
    print(f"Text: {text}")
    print("Predicted category:", label_map[pred])
    print("-" * 50)


Text: I have experience in neural networks and deep learning
Predicted category: AI / ML
--------------------------------------------------
Text: Skilled in pandas, numpy, and data visualization
Predicted category: Data Science
--------------------------------------------------
Text: Built REST APIs and backend services
Predicted category: AI / ML
--------------------------------------------------
Text: Developed responsive UI using HTML CSS and JavaScript
Predicted category: Web Development
--------------------------------------------------
